# STT

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [ ]:
import time
from utils.audio import Audio_record, Custom_faster_whisper
user_input = ""

In [ ]:
running = True # 모든 multi thread를 정지시키기 위한 플래그
model_name = 'base'
noiseValue = 0.1

# model, 음성 녹음 관련 초기화
audio_record = Audio_record()
model = Custom_faster_whisper()
model.set_model(model_name)

In [ ]:
# 녹음 시작
print("recording start")
audio_record.record_start()
while audio_record.recording:
    time.sleep(0.1)  # 100ms 대기
audio_dic = audio_record.record_stop(noiseValue)
print(audio_dic)
print("recording finish")

In [ ]:
# 모델 추론
_, result_denoise, time_denoise = model.run(audio_dic['audio_denoise'])

txt_denoise = f'{result_denoise} ({time_denoise}s)'
print(txt_denoise)
user_input = result_denoise

# LLM ChatBot

In [1]:
import threading
from utils.sLLM import Ollama_sLLM

In [2]:
# LLM 초기화
llm = Ollama_sLLM(model_name='llama3-ko-8b-Q8') # 언어 모델 설정
session_id = 'test4344332'
session_count = 0
llm.set_session_id(session_id)

Loaded in instruct/informations.txt


In [3]:
# 응답 관리 함수 정의
import re
import subprocess

def save_response(user_input, response):
    # 로그 저장
    log_entry = f"User: {user_input}\nChatBot: {response}\n--------\n"
    with open("./llm_log/chat_log.txt", "a", encoding="utf-8") as log_file:
        log_file.write(log_entry)
    
    extracted_code = extract_code_blocks(response)
    # 코드 저장
    if extracted_code: 
        extracted_code = "\n" + extracted_code
        with open("./llm_log/executed_code.txt", "a", encoding="utf-8") as code_file:
            code_file.write(extracted_code)  # 누적 저장
        with open("./llm_log/executing_code.py", "w", encoding="utf-8") as code_file:
            code_file.write(extracted_code)  # 실행할 코드
        # 현재 코드 실행
        run_executing_code()

# Function to extract code blocks from chatbot response and wrap each line in print()
def extract_code_blocks(response):
    code_blocks = re.findall(r'```(?:python)?\n(.*?)```', response, re.DOTALL)
    if not code_blocks:
        return None
    wrapped_code = []
    for block in code_blocks:
        lines = block.strip().split("\n")  # 코드 줄 단위로 분리
        if any("utils.log.show_executed_code()" in line for line in lines):
            wrapped_lines = [f'import sys\nimport os\nsys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), "..")))\nimport utils.log\n{line}' for line in lines]
        else:
            wrapped_lines = [f'print("{line}")' for line in lines]  # 각 줄을 print()로 감싸기
        wrapped_code.append("\n".join(wrapped_lines))

    return "\n\n".join(wrapped_code)

# Function to execute the generated Python script
def run_executing_code():
    try:
        result = subprocess.run(["python", "./llm_log/executing_code.py"], capture_output=True, text=True)
        print("[코드 실행 결과]\n", result.stdout)
        if result.stderr:
            print("\n실행 중 오류 발생\n", result.stderr)
    except Exception as e:
        print(f"실행 실패: {e}")

# Function to interact with the chatbot and save extracted code
def chat_with_bot(user_input):
    response = llm.invoke(user_input, session_id)
    save_response(user_input, response)
    return response


In [ ]:
# 현재 생성된 코드 실행
run_executing_code()

In [4]:
# 코드 초기화
import utils.log
utils.log.clear_executed_code()

실행된 코드 기록이 초기화되었습니다.


In [7]:
# 챗봇 테스트
user_input = "여기 연결된 CNC 문 열 수 있지?"
response = chat_with_bot(user_input)
print(f"User: {user_input}")
print(f"ChatBot: {response}")

User: 여기 연결된 CNC 문 열 수 있지?
ChatBot: 네, CNC 통신 모듈을 통해 문을 열 수 있습니다. 이대로 실행할까요? (코드)


In [ ]:
# 대화 초기화
session_count += 1
session_id += str(session_count)
llm.set_session_id(session_id)

# TTS

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
from utils.custom_tts import Custom_TTS

In [ ]:
tts_module = Custom_TTS()

In [ ]:
tts_module.set_model()

In [ ]:
import os
print(os.path.exists('tts_model/sample_sunhi.mp3')) # 44초 이상

In [ ]:
tts_module.get_reference_speaker(speaker_path='tts_model/sample_sunhi.mp3') # 여자 목소리
# tts_module.get_reference_speaker(speaker_path='sample_iena_3min.m4a') # 남자 목소리

In [ ]:
tts_module.make_speech(response)

In [ ]:
from IPython.display import Audio

# TTS된 음성 재생(목소리 변조 하기 이전)
Audio('./tts_output/tmp.wav')